In [6]:
#!pip install google-cloud-storage
# !pip install google-auth google-auth-oauthlib

  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     |████████████████████████████████| 151 kB 6.9 MB/s eta 0:00:01


In [1]:
from google.cloud import storage
from google.oauth2 import service_account

In [2]:
from google.cloud import bigquery
import os
from dotenv import load_dotenv
import pandas as pd


In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variable
credentials_file = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [5]:
client = storage.Client()

In [6]:
bucket_name = 'deployed-model-2023'
model_path = 'model.pkl'
blob = client.bucket(bucket_name).blob(model_path)

In [7]:
# stackoverflow

from google.cloud import storage

# Initialise a client
storage_client = storage.Client("sharedgcpproject")
# Create a bucket object for our bucket
bucket = storage_client.get_bucket(bucket_name)
# Create a blob object from the filepath
blob = bucket.blob("model.pkl")
# Download the file to a destination
destination_file_name = 'bucket_model.pkl'
blob.download_to_filename(destination_file_name)

In [8]:
import pickle

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [57]:
# local data 

data = pd.read_csv("clickbait_data.csv")

X_test = data.headline.sample(frac=0.2, random_state=42)
y_test = data.clickbait.sample(frac=0.2, random_state=42)

In [58]:
sum(y_test)

3273

In [59]:
y_pred = model.predict(X_test)

In [60]:
from sklearn.metrics import roc_auc_score

auroc = roc_auc_score(y_test, y_pred)
print('AUROC score:', auroc)

AUROC score: 0.9904774662517241


In [80]:
#!pip install --upgrade google-cloud-bigquery

In [15]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

query = """
    SELECT *
    From headline_dataset.source_data;
"""
query_job = client.query(query)  # Make an API request.

results = query_job.to_arrow()
# print("The query data:")
# for row in query_job:
#     print(row)

In [16]:
X_test = results.to_pandas().Headline

In [17]:
y_pred = model.predict(X_test)

In [18]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])

In [23]:
prediction = pd.DataFrame({'UUID':results.to_pandas().UUID, 'Clickbait': y_pred})

In [24]:
pd.set_option('display.max_rows', None)
prediction

,UUID,Clickbait
0,8f5b474a-3d74-4549-aef0-5e646b04a69f,0
1,a2afbd80-275a-469b-85a0-cdd7f30f8383,0
2,774ccd55-5af2-48cc-9f28-8f87edca559f,0
3,ea77ab17-27be-4755-99c2-aa4749c59ad8,0
4,425c9d2a-f10d-45a8-aeca-f07d19d5f3b8,0
5,3aeb0e97-f01b-4bbb-aa60-fa92650c7811,0
6,ceccec3b-3b2b-4d8d-9b27-5ae6e8ea250a,0
7,6c727831-7c21-4b48-a12d-94495202f82e,0
8,8da8ccc3-fdf7-4715-8086-06aa7d7bdaac,0
9,ab5c5764-503c-44eb-8caf-98e89e0c3a96,0


In [135]:
# import decimal
# # Initialize a BigQuery client object.
# client = bigquery.Client()

# # Define the ID of the existing table and the schema of the new column.
# table_id = 'headline_dataset.new_table'
# # schema = [bigquery.SchemaField('Headline', 'STRING'),
# #           bigquery.SchemaField('clickbait', 'INTEGER')]

# table = client.get_table(table_id)
# schema = table.schema


# # Load the data into a Pandas DataFrame and add the new column.
# table = results.to_pandas()
# table['clickbait'] = y_pred
# table['clickbait'] = table['clickbait'].astype(str).map(decimal.Decimal)


# table_id = 'headline_dataset.new_table'

# job_config = bigquery.LoadJobConfig(schema=schema)
# job = client.load_table_from_dataframe(table, table_id, job_config=job_config)

In [131]:
# table = client.get_table(table_id)
# original_schema = table.schema
# new_schema = original_schema[:]  # Creates a copy of the schema.
# new_schema.append(bigquery.SchemaField("clickbait", "NUMERIC"))

# table.schema = new_schema
# table = client.update_table(table, ["schema"])  # Make an API request.

In [27]:
table_id = 'headline_dataset.prediction'


schema = [
bigquery.SchemaField("UUID", "STRING"),
bigquery.SchemaField("Clickbait", "INT64"),
]



job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(prediction, table_id, job_config=job_config)